In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [ ]:
!nvidia-smi

In [4]:
!ldconfig -p | grep nccl

	libnccl.so.2 (libc6,x86-64) => /lib/x86_64-linux-gnu/libnccl.so.2
	libnccl.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libnccl.so


In [5]:
!python --version

Python 3.11.13


In [ ]:
import json
full_train_data = []
prompt = "Represent this sentence for searching relevant passages: "

with open("/kaggle/input/full-traindata-for-finetune-bge-m3-ver1/output_with_teacherscore_minedHN.jsonl", "r", encoding = "utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            temp_dict = json.loads(line)
            temp_dict['prompt'] = prompt
            full_train_data.append(temp_dict)

full_train_data[0]

In [7]:
import json

# Save to a regular JSON file (list of dicts)
with open("train_data.json", "w", encoding="utf-8") as f:
    json.dump(full_train_data, f, ensure_ascii=False, indent=2)

In [ ]:
!pip install -q -U FlagEmbedding[finetune]
!pip install -q -U faiss-cpu

In [9]:
!pip uninstall -y flash-attn

Found existing installation: flash_attn 2.8.2
Uninstalling flash_attn-2.8.2:
  Successfully uninstalled flash_attn-2.8.2


In [10]:
ds_stage0 = {
    "zero_optimization": {
      "stage": 0
    },
    
    # "fp16": {
    #   "enabled": True,
    #   "loss_scale": 0,
    #   "loss_scale_window": 1000,
    #   "initial_scale_power": 12,
    #   "hysteresis": 2,
    #   "min_loss_scale": 1
    # },
        "fp16": {
      "enabled": False,
    },
    "bf16": {
      "enabled": False
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupDecayLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto",
            "total_num_steps": "auto"
        }
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 100,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

with open("ds_stage0.json", "w") as f:
    json.dump(ds_stage0, f)

In [15]:
import torch
torch.cuda.empty_cache()

In [ ]:
import wandb
wandb.login(key="yourapikey")

In [ ]:
!nvidia-smi

Note:
    --passage_max_len 8192 \ #128 ok
    --save_steps 2000 \ #tang len neu dung nhieu epoch hon\ #--negatives_cross_device \ de xuong dong

In [ ]:
%%bash
torchrun --nproc_per_node 1 \
	-m FlagEmbedding.finetune.embedder.encoder_only.m3 \
	--model_name_or_path BAAI/bge-m3 \
    --cache_dir ./cache/model \
    --train_data /kaggle/working/train_data.json \
    --cache_path ./cache/data \
    --train_group_size 2 \
    --query_max_len 256 \
    --passage_max_len 512 \
    --pad_to_multiple_of 8 \
    --knowledge_distillation True \
    --same_dataset_within_batch True \
    --small_threshold 0 \
    --drop_threshold 0 \
    --output_dir ./test_encoder_only_m3_bge-m3_sd \
    --overwrite_output_dir \
    --learning_rate 1e-5 \
    --fp16 False\
    --num_train_epochs 1 \
    --per_device_train_batch_size 2 \
    --dataloader_drop_last True \
    --warmup_ratio 0.1 \
    --gradient_checkpointing \
    --deepspeed /kaggle/working/ds_stage0.json \
    --logging_steps 1 \
    --save_steps 5000 \
    --temperature 0.02 \
    --sentence_pooling_method cls \
    --normalize_embeddings True \
    --kd_loss_type m3_kd_loss \
    --unified_finetuning True \
    --use_self_distill True \
    --fix_encoder False \
    --self_distill_start_step 0

In [ ]:
# %%bash
# torchrun --nproc_per_node 2 \
# 	-m FlagEmbedding.finetune.embedder.encoder_only.base \
# 	--model_name_or_path "BAAI/bge-m3" \
#     --cache_dir ./cache/model \
#     --train_data /kaggle/working/train_data.json \
#     --cache_path ./cache/data \
#     --train_group_size 4 \
#     --query_max_len 256 \
#     --passage_max_len 8192 \
#     --pad_to_multiple_of 8 \
#     --query_instruction_for_retrieval 'Represent this sentence for searching relevant passages: ' \
#     --query_instruction_format '{}{}' \
#     --knowledge_distillation True \
# 	--output_dir ./test_encoder_only_base_bge-m3 \
#     --overwrite_output_dir \
#     --learning_rate 1e-5 \
#     --fp16 \
#     --num_train_epochs 1 \
#     --per_device_train_batch_size 1 \
#     --dataloader_drop_last True \
#     --warmup_ratio 0.1 \
#     --gradient_checkpointing \
#     --deepspeed /kaggle/working/ds_stage0.json \
#     --logging_steps 1 \
#     --save_steps 2000 \
#     --temperature 0.02 \
#     --sentence_pooling_method cls \
#     --normalize_embeddings True \
#     --kd_loss_type kl_div \

In [ ]:
%%bash
du -h --max-depth=1 /kaggle/working/test_encoder_only_m3_bge-m3_sd


In [ ]:
from FlagEmbedding import FlagModel

In [ ]:
finetuned_path_bge_m3 = "/kaggle/working/test_encoder_only_m3_bge-m3_sd/checkpoint-1095"

ft_model_bge_m3 = FlagModel(
    finetuned_path_bge_m3, 
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
    devices=[0],
    use_fp16=False
)

In [ ]:
finetuned_path_vnese_encoder = "/kaggle/input/vietnamese-bi-encoder-finetuned-v1/Checkpoint800"

ft_model_vnese_encoder = SentenceTransformer(
    model_name_or_path = finetuned_path_vnese_encoder,
    device = "cuda"
)

In [ ]:
i = 29

print(full_train_data[i]['pos_scores'])
print(full_train_data[i]['neg_scores'])

query_embedding = ft_model.encode(full_train_data[i]['query'])
positive_embedding = ft_model.encode(full_train_data[i]['pos'])
negative_embedding = ft_model.encode(full_train_data[i]['neg'])

similarity_1 = query_embedding @ positive_embedding.T
print(similarity_1)

similarity_2 = query_embedding @ negative_embedding.T
print(similarity_2)

In [ ]:
import json
with open("/kaggle/input/legal-corpus/legal_corpus.json", "r", encoding="utf-8") as f:
    data = json.load(f)

corpus = []
article_to_aid = {}
for law in data:
    for child_law in law['content']:
        article_to_aid[child_law['content_Article']] = child_law['aid']
        corpus.append(child_law['content_Article'])


Create index for fast searching


In [ ]:
submit_list = []
with open("/kaggle/input/public-test/public_test.json","r",encoding = "utf-8") as f:
    file = json.load(f)
questions = []
idx_to_qid = {}
for idx,q in enumerate(file):
    questions.append(q['question'])
    idx_to_qid[idx] = q['id']

In [ ]:
corpus_embeddings = ft_model.encode(corpus)

In [ ]:
import numpy as np

corpus_embeddings = corpus_embeddings.astype(np.float32)

In [ ]:
import faiss

# get the length of our embedding vectors
dim = corpus_embeddings.shape[-1]

# create the faiss index and store the corpus embeddings into the vector space
index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)

In [ ]:
index.train(corpus_embeddings)

In [ ]:
# check if the index is trained
print(index.is_trained)  

In [ ]:
index.add(corpus_embeddings)

print(f"total number of vectors: {index.ntotal}")

In [ ]:
len(questions)

In [ ]:
query_embeddings = ft_model.encode_queries(questions)

In [ ]:
for i in range(1,5):
    dists, ids = index.search(query_embeddings, k=i)
    submit = []
    for idx, each_ids in enumerate(ids):
        each_ids = [int(each_id) for each_id in each_ids]
        submit.append({"qid": idx_to_qid[idx], "relevant_laws": list(each_ids)})
        
    with open(f"answer_top{i}.json", "w", encoding="utf-8") as f:
        json.dump(submit, f, ensure_ascii=False)